In [1]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for langsmith: ")

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [2]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def devide(a: int, b: int) -> int:
    """devide a to b."""
    return a / b


tools = [add, multiply, devide]

llm_with_tools = llm.bind_tools(tools)

In [3]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x760524822300>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x76052483cc80>, root_client=<openai.OpenAI object at 0x760525391c40>, root_async_client=<openai.AsyncOpenAI object at 0x760524a68a10>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Adds a and b.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiplies a and b.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'devide', 'description': 'devide a to b.', 'parameters': {'properties': {'a': 

In [4]:
from langchain_core.messages import HumanMessage

query = "Hi. How are you today? What is 3.5 * 12.4? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_60WqGY0bPu5j3UOzwmjm0MO9', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_b7h9TSI0n8jaUs0EH32Q75Yg', 'type': 'tool_call'}]


In [5]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='Hi. How are you today? What is 3.5 * 12.4? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_60WqGY0bPu5j3UOzwmjm0MO9', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_b7h9TSI0n8jaUs0EH32Q75Yg', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 122, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVeQYn0njzqxioDl75Xh5OzoV3lyc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6b364206-2fb

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content="I'm good, thank you! \n\nThe result of \\(3.5 \\times 12.4\\) is approximately \\(36.4\\), and \\(11 + 49\\) equals \\(60\\).", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 188, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVeQaWNVWYlsl7OG1JgML7IZh2qTL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--04c3819c-77a7-47b5-b171-014c4dcddb1e-0', usage_metadata={'input_tokens': 188, 'output_tokens': 45, 'total_tokens': 233, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})